In [1]:
import os

In [7]:
import status_invest

In [24]:
resp_stocks_info = await status_invest.get_stocks_info()

In [40]:
stock = resp_stocks_info.json()[1]
stock

{'companyid': 252,
 'companyname': 'ESTACIO PARTICIPACOES S.A.',
 'ticker': 'YDUQ3',
 'price': 20.8,
 'p_l': 34.18,
 'dy': 1.32,
 'p_vp': 1.98,
 'p_ebit': 7.52,
 'p_ativo': 0.66,
 'ev_ebit': 12.39,
 'margembruta': 59.29,
 'margemebit': 17.03,
 'margemliquida': 3.75,
 'p_sr': 1.28,
 'p_capitalgiro': 7.38,
 'p_ativocirculante': -0.89,
 'giroativos': 0.51,
 'roe': 5.81,
 'roa': 1.92,
 'roic': 9.55,
 'dividaliquidapatrimonioliquido': 1.29,
 'dividaliquidaebit': 4.87,
 'pl_ativo': 0.33,
 'passivo_ativo': 0.67,
 'liquidezcorrente': 1.52,
 'peg_ratio': -0.07,
 'receitas_cagr5': 6.2,
 'lucros_cagr5': -15.03,
 'liquidezmediadiaria': 63657406.91,
 'vpa': 10.48,
 'lpa': 0.61,
 'valormercado': 6429048100.8,
 'segmentid': 26,
 'sectorid': 2,
 'subsectorid': 10,
 'subsectorname': 'Diversos',
 'segmentname': 'Serviços Educacionais',
 'sectorname': 'Consumo Cíclico'}

In [39]:

{
    'ativo': stock.get('ticker'),
    'empresa': stock.get('companyname'),
    'cotação': stock.get('price'),
    'p/l': stock.get('p_l'),
    'p/vp': stock.get('p_vp'),
    'psr': stock.get('p_sr'),
    'dividend yield': stock.get('dy'),
    'p/ativo': stock.get('p_ativo'),
    'p/capital de giro': stock.get('p_capitalgiro'),
    'p/ebit': stock.get('p_ebit'),
    'p/acl': stock.get('p_ativocirculante'),
    'ev/ebit': stock.get('ev_ebit'),
    'margem ebit': stock.get('margemebit'),
    'margem líquida': stock.get('margemliquida'),
    'liquidez corrente': stock.get('liquidezcorrente'),
    'roic': stock.get('roic'),
    'roe': stock.get('roe'),
    'cagr lucros 5 anos': stock.get('lucros_cagr5'),
    'cagr receitas 5 anos': stock.get('receitas_cagr5'),
    'dívida líquida/patrimônio': stock.get('dividaliquidapatrimonioliquido'),
    'dívida líquida/ebit': stock.get('dividaliquidaebit'),
    'roa': stock.get('roa'),
    'pl/ativos': stock.get('pl_ativo'),
    'giro ativos': stock.get('giroativos'),
    'margem bruta': stock.get('margembruta'),
    'passivo/ativo': stock.get('passivo_ativo'),
    'liquidez média diária': stock.get('liquidezmediadiaria')
}


{'ativo': 'YDUQ3',
 'empresa': 'ESTACIO PARTICIPACOES S.A.',
 'cotação': 20.8,
 'p/l': 34.18,
 'p/vp': 1.98,
 'psr': 1.28,
 'dividend yield': 1.32,
 'p/ativo': 0.66,
 'p/capital de giro': 7.38,
 'p/ebit': 7.52,
 'p/acl': -0.89,
 'ev/ebit': 12.39,
 'margem ebit': 17.03,
 'margem líquida': 3.75,
 'liquidez corrente': 1.52,
 'roic': 9.55,
 'roe': 5.81,
 'cagr lucros 5 anos': -15.03,
 'cagr receitas 5 anos': 6.2,
 'dívida líquida/patrimônio': 1.29,
 'dívida líquida/ebit': 4.87,
 'roa': 1.92,
 'pl/ativos': 0.33,
 'giro ativos': 0.51,
 'margem bruta': 59.29,
 'passivo/ativo': 0.67,
 'liquidez média diária': 63657406.91}

In [15]:
resp = await status_invest.get_stocks_historical_info('ZAMP3')
resp

('ZAMP3', <Response [200 OK]>)

In [27]:
resp[1].json()

TypeError: 'Response' object is not subscriptable

In [222]:
resp_page = await status_invest.get_stocks_page_info('YDUQ3')
resp_page

('YDUQ3', <Response [200 OK]>)

In [224]:
from bs4 import BeautifulSoup
from lxml import etree 

page = BeautifulSoup(resp_page[1].content, "html.parser") 
dom = etree.HTML(str(page)) 

In [284]:
def format_stock_page(resp_page):
    regex = re.compile('^[^\d\,%]*$')
    page = BeautifulSoup(resp_page[1].content, "html.parser") 
    dom = etree.HTML(str(page)) 
    elementos = dom.xpath('//div[@id="company-section"]//div[@class="info"]')
    result = {}
    tag = ''
    for elemento in elementos:
        children = elemento.xpath('./div//div//strong|./div//div//h3|./div//div//span[@class="d-inline-block mr-2"]')
        for child in children:
            if child.text and not child.attrib['class'] == 'title m-0 legend-tooltip':
                if child.tag in {'h3', 'span'}:
                    tag = child.text
                else:
                    result[tag] = child.text
                    if not regex.match(child.text):
                        print(child.text)
                        result[tag] = float(child.text.replace('.', '').replace('%', '').replace(',', '.'))

                    
    elementos_setor = dom.xpath('//div[@id="company-section"]//div[@class="top-info top-info-1 top-info-sm-2 top-info-md-n sm d-flex justify-between"]//div[contains(@class, "info")]')
    for elemento in elementos_setor:
        children = elemento.xpath('./div//div//strong|./div//div//span')
        for child in children:
            if child.text and not child.attrib['class'] == 'title m-0 legend-tooltip':
                if child.tag in {'h3', 'span'}:
                    tag = child.text
                else:
                    result[tag] = child.text
    return result

resp_page = await status_invest.get_stocks_page_info('VALE3')
format_stock_page(resp_page)

190.172.000.000
445.621.000.000
73.474.000.000
70.289.000.000
20.168.000.000
50.121.000.000
338.836.915.847
388.957.915.847
4.539.007.580
98,01%


{'Patrimônio líquido': 190172000000.0,
 'Ativos': 445621000000.0,
 'Ativo circulante': 73474000000.0,
 'Dívida bruta': 70289000000.0,
 'Disponibilidade': 20168000000.0,
 'Dívida líquida': 50121000000.0,
 'Valor de mercado': 338836915847.0,
 'Valor de firma': 388957915847.0,
 'Nº total de papéis': 4539007580.0,
 'Segmento de listagem': 'Novo Mercado',
 'Free Float': 98.01,
 'Setor de Atuação': 'Materiais Básicos',
 'Subsetor de Atuação': 'Mineração',
 'Segmento de Atuação': 'Minerais Metálicos'}

In [313]:
import re
regex = re.compile('^[-\d\,%\.]*$')
regex.match('-98,01')



<re.Match object; span=(0, 6), match='-98,01'>

In [294]:
# 10592034101
# 12,39
# EV/EBIT = 12,39
# 10592034101/EBIT = 12,39
10592034101 / 12.39
round((854885722.43 / 10592034101) * 100, 2)
    

8.07